Variables:

- flag superficie imputada
- 

In [197]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from malbecs.modeling import train as tr

wine_path = "../../data/final/wine_final.csv"
eto_path = "../../data/final/eto_final.csv"
meteo_path = "../../data/final/meteo_final.csv"

def show_feat_imps(feat_imp, feat_names):
    pd.DataFrame(
        feat_imp,
        index=feat_names,
        columns=["feat_imp"]
    ).sort_values("feat_imp")[-50:].plot(kind='barh', figsize=(6, 15))


In [198]:

data = tr.load_final_data(
    wine_path=wine_path,
    eto_path=eto_path,
    meteo_path=meteo_path
)

with open("../../data/final/meteo_features.txt", "r", encoding="utf-8") as f:
    meteo_cols = f.read().split("\n")

with open("../../data/final/eto_features.txt", "r") as f:
    eto_cols = f.read().split("\n")

with open("../../data/final/wine_features.txt", "r") as f:
    wine_cols = f.read().split("\n")


In [199]:
data_train = tr.filter_camp(data.copy(), min_camp=15, max_camp=21)

data_final = tr.filter_camp(data, min_camp=22, max_camp=22)

train, test = tr.train_test_split(data_train, test_camp=21)

X, y = tr.xy_split(data_train)

cat_cols = [
    'id_finca',
    'id_zona',
    'id_estacion',
    'variedad',
    "modo",
    "tipo",
    "color",
    "prod_shift1_gt_shift2",
    "sup_is_nan",
]
num_cols = [col for col in X.columns if col not in cat_cols]

X[cat_cols] = X[cat_cols].astype('category')

X_train, y_train = tr.xy_split(train)
X_test, y_test = tr.xy_split(test)
X_final, y_final = tr.xy_split(data_final)

train_idxs, test_idxs = tr.CampKFold.get_train_test(
    X['campaña'], from_camp=19, to_camp=21
)

cv = tr.CampKFold(train_idxs, test_idxs)


In [203]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import KBinsDiscretizer, OrdinalEncoder, StandardScaler
import pickle as pkl
from typing import List
import malbecs.modeling.transformers as mt

seed = 42

def get_base_model():

    model_num_cols = [
        'superficie',
        'prod_shift_max',
        'prod_shift_change',
        'prod_shift_avg',
    ]

    m = make_pipeline(
        make_column_transformer(

            (mt.BaseNEncoder(), ['id_finca']),

            (mt.TargetEncoder(), ['id_zona']),

            (OrdinalEncoder(handle_unknown='use_encoded_value',
             unknown_value=-1), ['id_estacion']),

            (mt.BaseNEncoder(), ['variedad']),

            (OrdinalEncoder(handle_unknown='use_encoded_value',
             unknown_value=-1), ['modo']),

            (KBinsDiscretizer(n_bins=2, random_state=seed), ['altitud']),

            (StandardScaler(), model_num_cols),

            remainder='drop'
        ),
        RandomForestRegressor(
            random_state=seed,
            n_estimators=200,
            min_samples_leaf=4,
            n_jobs=-1,
            max_features='sqrt',
            max_samples=0.8
        )
    )
    return m


In [204]:
m = get_base_model()

In [205]:
from sklearn.model_selection import cross_validate
import numpy as np

res = cross_validate(
    estimator=m,
    X=X,
    y=y,
    cv=cv,
    n_jobs=-1,
    scoring=tr.rmse_scorer,
    return_train_score=True,
    return_estimator=True
)


print("train: ", res['train_score'])
print("test: ", res['test_score'])
print(f"Train Mean RMSE: {np.mean(res.get('train_score'))}")
print(f"Test Mean RMSE: {np.mean(res.get('test_score'))}")

train:  [-4950.42915184 -4769.7101927  -4818.80517865]
test:  [-4867.31956362 -6767.87298344 -5430.5269726 ]
Train Mean RMSE: -4846.314841064502
Test Mean RMSE: -5688.573173223823


In [8]:
percip_cols = [c for c in eto_cols if "Precip" in c]
snow_cols = [c for c in eto_cols if "Snow" in c]
temp_day_cols = [c for c in eto_cols if "TemperatureLocalDay" in c]
temp_cols = [
    c for c in eto_cols 
    if "TemperatureLocalAfter" in c or "TemperatureLocalOvern" in c
]
evotrans_cols = [c for c in eto_cols if "Evapotranspiration" in c]
feelslike_cols = [c for c in eto_cols if "FeelsLikeLoca" in c]
irrad_cols = [c for c in eto_cols if "Irradiance" in c]
gust_cols = [c for c in eto_cols if "Gust" in c]
wind_cols = [c for c in eto_cols if "Wind" in c]
dewpoint_cols = [c for c in eto_cols if "Dewpoint" in c]
mslp_cols = [c for c in eto_cols if "MSLP" in c]
humid_cols = [c for c in eto_cols if "Humidity" in c]
uvindex_cols = [c for c in eto_cols if "UVIndex" in c]
visib_cols = [c for c in eto_cols if "Visibility" in c]

In [9]:
tempuo = [c for c in eto_cols if ("Temp" in c) and ("StdM" in c )]
precipuo = [c for c in eto_cols if ("Precip" in c) and ("StdM" in c )]
snowpuo = [c for c in eto_cols if ("Snow" in c) and ("StdM" in c )]
winduo = [c for c in eto_cols if ("Wind" in c) and ("StdM" in c )]
gustuo = [c for c in eto_cols if ("Gust" in c) and ("StdM" in c)]

In [10]:
# n_fincas = X['id_finca'].nunique()
# top_fincas = X.id_finca.value_counts()[:int(0.8*n_fincas)].index.values.tolist()
# X['id_finca_top'] = X['id_finca'].apply(lambda x: x if x in top_fincas else -1)

# n_zonas = X['id_zona'].nunique()
# top_zonas = X['id_zona'].value_counts()[:int(0.8*n_zonas)].index.values.tolist()
# X['id_zona_top'] = X['id_zona'].apply(lambda x: x if x in top_zonas else -1)

# n_var = X['variedad'].nunique()
# top_variedad= X['variedad'].value_counts()[:int(0.8*n_var)].index.values.tolist()
# X['variedad_top'] = X['variedad'].apply(
#     lambda x: x if x in top_variedad else -1)

In [11]:

# mt.QuantileFeatureEncoder(qs=np.linspace(0.1, 0.9, 9).tolist(), col='id_finca').fit_transform(X,y)['id_finca'].value_counts()
# X.filter(like='sup')
# X[[c for c in meteo_cols if "avg" in c ]].hist(figsize=(15,15))
# from sklearn.decomposition import PCA

# cols_pca = [c for c in gust_cols if 'Max' in c]
# pca = make_pipeline(StandardScaler(),PCA(random_state=seed))


# wine_num_cols

# X[wind_cols].hist(figsize=(15,15))
# X[[c for c in humid_cols if "Avg" in c]].hist(figsize=(15,15))
# from sklearn.decomposition import PCA, KernelPCA
# pca = make_pipeline(StandardScaler(),PCA())

# Xt = pca.fit_transform(X[cols_pca])


# plot = plt.scatter(Xt[:,0], Xt[:,1])

# plt.show()

# plt.plot(
#     range(1,pca[-1].explained_variance_ratio_.shape[0]+1),
#     np.cumsum(pca[-1].explained_variance_ratio_),
# )
# temp_cols
# [c for c in snowpuo if "Month1" in c or "Month2" in c]


# X[precipuo].hist(figsize=(15,15))

In [19]:
# from sklearn.base import BaseEstimator, OneToOneFeatureMixin, TransformerMixin


# class GroupedQuantileFeatureEncoder(BaseEstimator, TransformerMixin, OneToOneFeatureMixin):

#     def __init__(self, col: str, groups:List[str], value:str, qs=[0.25, 0.5, 0.75], scale=True):
#         self.col = col
#         self.groups = groups
#         self.value = value
#         self.qs = qs
#         self.scale = scale
        

#     def fit(self, X: pd.DataFrame, y: pd.Series):

#         X = X.copy()

#         if not isinstance(X, pd.DataFrame):
#             return Exception("X must be of type pd.DataFrame")
#         if not isinstance(y, pd.Series):
#             return Exception("y must be of type pd.Series")

    
#         self.category_means_ = X.groupby(self.groups)[self.value].mean()
#         # category_means_ = pd.concat([X[self.col], y], axis=1).groupby(self.col)[
#             # y.name].mean()

#         self.qs_ = [self.category_means_.quantile(q) for q in self.qs]

#         def encode_qs(x, qs):
#             for i, q in enumerate(qs):
#                 if x < q:
#                     return i
#             return len(qs)

#         self.category_encodings_ = self.category_means_.apply(
#             lambda x: encode_qs(x, self.qs_)
#         ).to_dict()

#         return self

#     def transform(self, X):
#         X = X.copy()
#         X.groupby(self.groups)[self.col].map(self.category_encodings_)
#         # X[self.col] = X[self.col].map(self.category_encodings_
#         X[self.col] = X[self.col].fillna(-1)
#         return X
    

# gqe = GroupedQuantileFeatureEncoder(
#     col="id_finca",
#     groups=["variedad","modo"],
#     value="prod_he_shift1"
# )

# gqe.fit(X,y)

# # gqe.category_encodings_
# # gqe.mean_
# # X['prod_he_shift1']
# xcopy = X.copy()
# # xcopy['a'] = 
# xcopy.set_index(gqe.groups).index.map(gqe.category_encodings_)
# # xcopy['a']
# # .transform(map(gqe.category_encodings_))
# # gqe.category_encodings_
# # gqe.category_encodings_
# # gqe.category_means_
# gqe.qs_

# # X.groupby(["variedad","campaña"])["prod_he_shift1"].mean()
# # X.groupby(["id_finca","campaña"])["prod_he_shift1"].mean()
# # X.groupby(["id_estacion","campaña"])["prod_he_shift1"].mean()
# # X['prod_he_shift1']

In [20]:
# X.groupby(["id_finca","campaña"])['prod_he_shift1'].quantile(0.25)

# X.groupby(["variedad","modo"])['prod_he_shift1'].quantile(0.9)
# X.groupby(["variedad","modo"])['prod_he_shift1'].quantile(0.25)

# X.set_index(['id_finca','variedad',"modo"])['prod_he_shift1']


In [485]:
from sklearn.decomposition import PCA


def get_preprocesing():

    model_num_cols = [
        'altitud',
        'campaña',
        'superficie',

        "prod_shift1",
        'prod_shift_max',
        'prod_shift_change',
        'prod_shift_avg',

        'prod_he_var_zone_mean_hist_total',
        'prod_he_var_zone_std_hist_total',

        'prod_he_var_modo_zona_mean_shift1_total',
        "prod_he_var_modo_zona_change_total",

        "prod_he_var_mean_shift1_total",
        "prod_he_var_change_total",

        "prod_he_var_modo_mean_shift1_total",
        "prod_he_var_modo_change_total",

    ]

    return make_column_transformer(

        (OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1), ['sup_is_nan']),

        # (mt.BaseNEncoder(), ['id_finca']),
        # (mt.TargetEncoder(), ['id_finca']),
        # (mt.QuantileFeatureEncoder(col='id_finca'), ['id_finca']),
        # (OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), ['id_finca']),
        # (OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1), ['id_finca_top']),
         
        # (mt.TargetEncoder(), ['id_zona']),
        (OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1), ['id_zona']),

        # (OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1), ['id_estacion']),

        # (mt.BaseNEncoder(), ['variedad']), 
        (OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1), ['variedad']),
        # (mt.QuantileFeatureEncoder(qs=np.linspace(0.1, 0.9, 9).tolist(), col='variedad'), ['variedad']),

        (OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1), ['modo']),

        # (KBinsDiscretizer(n_bins=3, encode='ordinal'), ['altitud']),
        # (KBinsDiscretizer(n_bins=5, encode='ordinal'), ['prod_shift2']),

        (StandardScaler(), model_num_cols),
        
        # (StandardScaler(),temp_cols),

        # (make_pipeline(StandardScaler(),PCA(n_components=3, random_state=seed)), temp_cols),
        
        # (KBinsDiscretizer(n_bins=2), [c for c in temp_day_cols if "Avg" in c]),

        # (StandardScaler(), [c for c in temp_day_cols if "Avg" in c]),
        # (make_pipeline(StandardScaler(),PCA(n_components=2, random_state=seed)), [c for c in temp_day_cols if "Avg" in c]),

        # (StandardScaler(),[c for c in meteo_cols if "avg_daytime" in c]),

        # (make_pipeline(StandardScaler(), PCA(n_components=2, random_state=seed)), [
        #  c for c in meteo_cols if "avg_daytime" in c]),
        
        
        # (StandardScaler(),precipuo),
        # (StandardScaler(),tempuo),
        # (StandardScaler(),winduo),
        # (StandardScaler(),gustuo),
        # (StandardScaler(),[c for c in snowpuo if "Month1" in c or "Month2" in c or "Month3" in c])

        (StandardScaler(),[c for c in precipuo if "2Std" in c]),
        # (StandardScaler(), [c for c in precipuo if "1Std" in c]),
        # (StandardScaler(),[c for c in winduo if "2Std" in c]),
        # (StandardScaler(),[c for c in tempuo if "2Std" in c]),
        # (StandardScaler(),[c for c in gustuo if "2Std" in c]),
        # (StandardScaler(),[c for c in snowpuo if "Month1" in c or "Month2" in c or "Month3" in c]),

        # (make_pipeline(StandardScaler(),PCA(n_components=2, random_state=seed)),
            #  [c for c in percip_cols if 'Sum' in c]
        # ),

        (StandardScaler(),[c for c in snow_cols if 'Sum' in c and ("1" in c or "2" in c)]),
        
        # (StandardScaler(), [c for c in gust_cols if 'Max' in c ]),

        # (make_pipeline(StandardScaler(),PCA(n_components=2, random_state=seed)), [c for c in gust_cols if 'Max' in c ]),

        # (StandardScaler(),[c for c in humid_cols if "Avg" in c]),
        # (make_pipeline(StandardScaler(),PCA(n_components=2, random_state=seed)), [c for c in humid_cols if "Avg" in c]),
        # (make_pipeline(StandardScaler(),PCA(n_components=3, random_state=seed)), uvindex_cols),
        # (make_pipeline(StandardScaler(),PCA(n_components=3, random_state=seed)), dewpoint_cols),
        # (make_pipeline(StandardScaler(),PCA(n_components=2, random_state=seed)), wind_cols),


        remainder='drop'
    )


In [502]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor, VotingRegressor
import xgboost as xgb

prep = get_preprocesing()

model = RandomForestRegressor(
    random_state=seed,
    n_estimators=200,
    min_samples_leaf=4,
    # max_depth=8,
    n_jobs=-1,
    # max_features=0.22,
    max_features="sqrt",
    max_samples=0.8
    # random_state=seed,
    # n_estimators=400,
    # n_jobs=-1,
    # max_features=1.0,
    # max_samples=1.0
)

model = xgb.XGBRegressor(
    # tree_method="exact",
    learning_rate=0.009261187281287938,
    random_state = seed,
    n_estimators = 840,
    subsample=0.8,
    colsample_bytree = 0.22,
    colsample_bynode = 0.70,
    # colsample_bylevel = 0.20,
    max_depth=6,
    # reg_alpha=100,
    reg_lambda=22,
    n_jobs=-1,
)

est_xgb = lambda i: xgb.XGBRegressor(
    # tree_method="exact",
    learning_rate=0.009261187281287938,
    random_state=seed+i,
    n_estimators=840,
    subsample=0.8,
    colsample_bytree=0.22,
    colsample_bynode=0.70,
    # colsample_bylevel = 0.20,
    max_depth=6,
    # reg_alpha=100,
    reg_lambda=22,
    n_jobs=-1,
)
est_rf = lambda i: RandomForestRegressor(
    random_state=seed+i,
    n_estimators=180,
    min_samples_leaf=4,
    # max_depth=8,
    n_jobs=-1,
    # max_features=0.22,
    max_features="sqrt",
    max_samples=0.8
    # random_state=seed,
    # n_estimators=400,
    # n_jobs=-1,
    # max_features=1.0,
    # max_samples=1.0
)

model = VotingRegressor(
    estimators=[(f"est_{i}", est_xgb(i)) for i in range(5)],
)

# model = StackingRegressor(
#     estimators=[("est1",est_xgb(1)), ("est2",est_xgb(0)),("est3",est_rf(0))],
#     final_estimator=make_pipeline(StandardScaler(), LinearRegression()),
# )

m = make_pipeline(
    prep,
    model
)

In [503]:

# sample_weight = X['campaña'].apply(lambda x: 0.5 if x == 20 else 1).values
# sample_weight = X['campaña'].apply(lambda x: x/22)
# sample_weight = X['campaña'].apply(lambda x: 1 if x==20 else 0.5)
# sample_weight = X['campaña'].apply(lambda x: 0.2 if x <= 19 else 1)


res = cross_validate(
    estimator=m,
    X=X,
    y=y,
    cv=cv,
    n_jobs=-1,
    scoring=tr.rmse_scorer,
    return_train_score=True,
    return_estimator=True,
    fit_params={
        # "randomforestregressor__sample_weight": sample_weight
        # "xgbregressor__sample_weight": sample_weight
        # "stackingregressor__sample_weight": sample_weight
    }
)

print("train: ", res['train_score'])
print("test: ", res['test_score'])
print(f"Train Mean RMSE: {np.mean(res.get('train_score'))}")
print(f"Test Mean RMSE: {np.mean(res.get('test_score'))}")


# base
# train:  [-4950.42915184 - 4769.7101927 - 4818.80517865]
# test:  [-4867.31956362 - 6767.87298344 - 5430.5269726]
# Train Mean RMSE: -4846.314841064502
# Test Mean RMSE: -5688.573173223823

# train:  [-4999.3034144 - 4828.81703503 - 4918.20715021]
# test:  [-4844.22108053 - 6787.95522349 - 5346.60658675]
# Train Mean RMSE: -4915.442533217256
# Test Mean RMSE: -5659.594296923198


train:  [-4969.85670136 -4802.25367454 -4909.09924514]
test:  [-4837.82060266 -6812.07541638 -5381.24129592]
Train Mean RMSE: -4893.736540345112
Test Mean RMSE: -5677.045771655104


In [469]:
from numpy import linspace
from sklearn.model_selection import GridSearchCV

param_grid = {
    
    # 'randomforestregressor__max_depth': [4, 5, 6, 8],
    # 'randomforestregressor__ccp_alpha': [0,0.01,0.02],


    # 'randomforestregressor__max_features': [0.22],
    # 'randomforestregressor__max_samples': [0.8], 
    # 'randomforestregressor__min_samples_leaf':[4],
    # 'randomforestregressor__n_estimators': [180,200,300,400,500],

    # 'xgbregressor__max_features': [0.22],
    # 'xgbregressor__max_samples': [0.8],
    # 'xgbregressor__min_samples_leaf': [4],
    'xgbregressor__n_estimators': range(820,860,10),
    # 'xgbregressor__learning_rate': np.logspace(-2.07, -2, 5), #np.logspace(-2.07, -1.9, 10)
}

gsm = GridSearchCV(
    m,
    param_grid=param_grid,
    cv=cv,
    verbose=1,
    scoring=tr.rmse_scorer
)

gsm.fit(
    X, 
    y, 
    # randomforestregressor__sample_weight=sample_weight
    )

print(gsm.best_params_)
print(gsm.best_score_)


Fitting 3 folds for each of 4 candidates, totalling 12 fits
{'xgbregressor__n_estimators': 840}
-5659.341004910268


In [471]:
print(gsm.best_params_)
print(gsm.best_score_)

# {'xgbregressor__learning_rate': 0.009261187281287938}
# -5659.594296923198


{'xgbregressor__n_estimators': 840}
-5659.341004910268


In [470]:
# m = gsm.best_estimator_
# y_pred = m.fit(X_train, y_train).predict(X_test)
# score_model(X_test, y_test, y_pred, sup_norm=False)

res = cross_validate(
    estimator=gsm.best_estimator_,
    X=X,
    y=y,
    cv=cv,
    n_jobs=-1,
    scoring=tr.rmse_scorer,
    return_train_score=True,
    return_estimator=True,
    # fit_params={
    #     "randomforestregressor__sample_weight": sample_weight
    # }
)

print("train: ", res['train_score'])
print("test: ", res['test_score'])
print(f"Train Mean RMSE: {np.mean(res.get('train_score'))}")
print(f"Test Mean RMSE: {np.mean(res.get('test_score'))}")


train:  [-4984.0148598  -4812.46729515 -4903.08592288]
test:  [-4847.9543898  -6785.57169828 -5344.49692665]
Train Mean RMSE: -4899.85602594488
Test Mean RMSE: -5659.341004910268
